In [1]:
import pandas as pd
import mysql.connector

**Conexion a la BD**

In [2]:
db_config = {
    "host": "localhost",
    "user":"prueba",
    "password":"Mero_8439*",
    "database":"PRUEBA"
}

try:
    conexion = mysql.connector.connect(**db_config)
    cursor = conexion.cursor()
    print("Conexión exitosa a MySQL")
except mysql.connector.Error as err:
    print(f"Error al conectar a MySQL: {err}")

Conexión exitosa a MySQL


**Carga de datos**

In [ ]:
df_asignatura = pd.read_csv("Asignaturas.csv")

print("Columnas en asignatura.csv:", df_asignatura.columns)

for index, row in df_asignatura.iterrows():
    query = "INSERT INTO asignaturas (asignatura_id, nombre, semestre) VALUES (%s, %s, %s)"
    values = (row['asignatura_id'], row['nombre'], row['Semestre'])
    try:
        cursor.execute(query, values)
    except mysql.connector.Error as err:
        print(f"Error al insertar en asignaturas: {err}")

conexion.commit() 

Columnas en asignatura.csv: Index(['asignatura_id', 'nombre', 'Semestre'], dtype='object')


In [ ]:
df_estudiante = pd.read_csv("estudiantes.csv")
print("Columnas en estudiante.csv:", df_estudiante.columns)
for index, row in df_estudiante.iterrows():
    query = "INSERT INTO estudiante (numero_cuenta, nombre) VALUES (%s, %s)"
    values = (row['numero_cuenta'], row['nombre'])
    try:
        cursor.execute(query, values)
    except mysql.connector.Error as err:
        print(f"Error al insertar en estudiante: {err}")

conexion.commit()

In [17]:
df_inscripcion = pd.read_csv("Inscripcion.csv")
print("Columnas en inscripcion.csv:", df_inscripcion.columns)

for index, row in df_inscripcion.iterrows():
    query = "INSERT INTO inscripcion (curso_id, calificacion, asignatura_id, numero_cuenta) VALUES (%s, %s, %s, %s)"

    # Convertir a tipos nativos de Python
    curso_id = int(row['curso_id'])  
    calificacion = float(row['calificacion'])  # Si puede contener decimales, usa float
    asignatura_id = int(row['asignatura_id'])
    num_cuenta = int(row['num_cuenta'])

    values = (curso_id, calificacion, asignatura_id, num_cuenta)

    try:
        cursor.execute(query, values)
    except mysql.connector.Error as err:
        print(f"Error al insertar en inscripcion: {err}")

conexion.commit()

Columnas en inscripcion.csv: Index(['curso_id', 'calificacion', 'num_cuenta', 'asignatura_id'], dtype='object')


**Realice una consulta que muestre las tres calificaciones mas altas obtenidas en cada asignatura.En la misma salida muestre el promedio de cada materia y una bandera que compare esta
calificación con la obtenida por los alumnos en la primera consulta donde ‘1’ significa que la
calificación fue mas alta que el promedio y ‘0’ que fue más baja.
Con el uso de Python guarde en un CSV a los alumnos que tienen una calificación mayor al
promedio.**

In [ ]:
try:
    query = """
    SELECT
        a.nombre AS asignatura,
        e.nombre AS estudiante,
        i.calificacion,
        (SELECT AVG(calificacion) FROM inscripcion WHERE asignatura_id = i.asignatura_id) AS promedio,
        IF(i.calificacion > (SELECT AVG(calificacion) FROM inscripcion WHERE asignatura_id = i.asignatura_id), 1, 0) AS bandera
    FROM
        (SELECT
             asignatura_id,
             numero_cuenta,
             calificacion,
             @row_number := IF(@asignatura_id = asignatura_id, @row_number + 1, 1) AS rn,
             @asignatura_id := asignatura_id
         FROM
             inscripcion
         ORDER BY
             asignatura_id, calificacion DESC
        ) i
    JOIN
        asignaturas a ON i.asignatura_id = a.asignatura_id
    JOIN
        estudiante e ON i.numero_cuenta = e.numero_cuenta
    WHERE
        i.rn <= 3
    ORDER BY
        a.nombre, i.calificacion DESC;
    """

    # Inicializar las variables de sesión
    cursor.execute("SET @row_number = 0, @asignatura_id = NULL;")

    # Ejecutar el query
    cursor.execute(query)

    # Obtener los resultados
    resultados = cursor.fetchall()

    # Convertir los resultados a un DataFrame de pandas
    df_resultados = pd.DataFrame(resultados, columns=['asignatura', 'estudiante', 'calificacion', 'promedio', 'bandera'])

    # Imprimir el DataFrame
    print(df_resultados)

    # Filtrar alumnos con calificación mayor al promedio
    alumnos_mayor_promedio = df_resultados[df_resultados['bandera'] == 1]

    # Guardar en CSV
    alumnos_mayor_promedio.to_csv('alumnos_mayor_promedio.csv', index=False)

    print("Alumnos con calificación mayor al promedio guardados en alumnos_mayor_promedio.csv")

except mysql.connector.Error as err:
    print(f"Error al ejecutar el query: {err}")

                      asignatura                estudiante calificacion  \
0    Administracion de Proyectos  Vania Castellanos Juarez        10.00   
1    Administracion de Proyectos       Nico Verdugo Blanes         9.00   
2    Administracion de Proyectos      David Romero Mendoza         9.00   
3                       Algebra          Fanny Rojas Tello        10.00   
4                       Algebra          Javier Cruz Rojas        10.00   
..                           ...                       ...          ...   
116          Sistemas Operativos       Nico Verdugo Blanes        10.00   
117          Sistemas Operativos       Carlos Juarez David        10.00   
118                Termodinamica            Rosalva Vargas        10.00   
119                Termodinamica    Maria Barriga Sandoval        10.00   
120                Termodinamica  Vania Castellanos Juarez        10.00   

     promedio  bandera  
0    7.153846        1  
1    7.153846        1  
2    7.153846        1  

**Ejercicio 2**
1. Realice una consulta que muestre el porcentaje de avance de créditos de cada alumno con
una bandera que clasifique a los alumnos de la siguiente manera:
- Bandera ‘5’: Alumnos que tienen el 100% de créditos
- Bandera ‘4’: Alumnos que tienen entre >=80% y <100% de créditos
- Bandera ‘3’: Alumnos que tienen entre >=65% y <80% de créditos
- Bandera ‘2’: Alumnos que tienen entre >=40% y <65% de créditos
- Bandera ‘1’: Alumnos que tienen entre >=0% y <40% de créditos
2. Con el uso de Python obtenga el top 10 y el bottom 10 de los alumnos con mayor y menor
avance de créditos
3. Genere un CSV bajo los siguientes lineamientos
Almacene solo el nombre de los 10 alumnos con el mayor avance de créditos
Almacene solo el nombre de los 10 alumnos con el menor avance de créditos

In [ ]:
try:
    conexion = mysql.connector.connect(**db_config)
    cursor = conexion.cursor()
    print("Conexión exitosa a MySQL")

    query = """
        SELECT
            e.nombre AS nombre_alumno,
            COUNT(i.asignatura_id) AS total_materias_cursadas,
            (SELECT COUNT(*) FROM asignaturas) AS total_materias,
            (COUNT(i.asignatura_id) / (SELECT COUNT(*) FROM asignaturas)) * 100 AS porcentaje_avance,
            CASE
                WHEN (COUNT(i.asignatura_id) / (SELECT COUNT(*) FROM asignaturas)) * 100 = 100 THEN 5
                WHEN (COUNT(i.asignatura_id) / (SELECT COUNT(*) FROM asignaturas)) * 100 >= 80 THEN 4
                WHEN (COUNT(i.asignatura_id) / (SELECT COUNT(*) FROM asignaturas)) * 100 >= 65 THEN 3
                WHEN (COUNT(i.asignatura_id) / (SELECT COUNT(*) FROM asignaturas)) * 100 >= 40 THEN 2
                ELSE 1
            END AS bandera_clasificacion
        FROM
            inscripcion i
        JOIN
            estudiante e ON i.numero_cuenta = e.numero_cuenta
        GROUP BY
            e.nombre
        ORDER BY
            porcentaje_avance DESC;
    """
    cursor.execute(query)
    resultados = cursor.fetchall()

    df = pd.DataFrame(resultados, columns=['nombre_alumno', 'total_materias_cursadas', 'total_materias', 'porcentaje_avance', 'bandera_clasificacion'])

    top_10 = df.head(10)
    bottom_10 = df.tail(10)

    top_10.to_csv('top_10_alumnos.csv', index=False)
    bottom_10.to_csv('bottom_10_alumnos.csv', index=False)

    print("Top 10 alumnos guardados en top_10_alumnos.csv")
    print("Bottom 10 alumnos guardados en bottom_10_alumnos.csv")


except mysql.connector.Error as err:
    print(f"Error al conectar a MySQL: {err}")
finally:
    if 'conexion' in locals() and conexion.is_connected():
        cursor.close()
        conexion.close()
        print("Conexión cerrada")


Conexión exitosa a MySQL
Top 10 alumnos guardados en top_10_alumnos.csv
Bottom 10 alumnos guardados en bottom_10_alumnos.csv
Conexión cerrada


In [26]:
print(top_10.head(10))

              nombre_alumno  total_materias_cursadas  total_materias  \
0           Kike Vicens Bou                       40              51   
1            Rosalva Vargas                       39              51   
2       Nico Verdugo Blanes                       39              51   
3      David Romero Mendoza                       38              51   
4            Daiara Saldana                       38              51   
5       Jorge Jimenez Palma                       38              51   
6          Alexdr Nikolenko                       38              51   
7          Karina Matamoroz                       38              51   
8  Vania Castellanos Juarez                       38              51   
9         Diego Luz Alvares                       38              51   

  porcentaje_avance  bandera_clasificacion  
0           78.4314                      3  
1           76.4706                      3  
2           76.4706                      3  
3           74.5098        

In [27]:
print(bottom_10.head(10))

                         nombre_alumno  total_materias_cursadas  \
21         Alexandra Navarrete Ramirez                       26   
22  Rafael Alejandro Ramirez Contreras                       26   
23              Paulina Arino Portillo                       26   
24                   Lupita de Esteban                       26   
25               Calisto Bolanos Bravo                       15   
26                 Eugenio Galan Giron                       15   
27              Hugo Martinez Martinez                       15   
28       Juan Jose Ballesteros Aguirre                       10   
29              Adelina Garcia Sanchez                       10   
30                       Juliana Jodar                        5   

    total_materias porcentaje_avance  bandera_clasificacion  
21              51           50.9804                      2  
22              51           50.9804                      2  
23              51           50.9804                      2  
24            

**Pregunta 3:**

Explique cual sería el procedimiento para cargar los archivos CSV generados en el ejercicio anterior
en una base de datos mediante el uso de alguna herramienta ETL que usted conozca

**Para Power BI**

- 1.- Abrir Power BI Desktop
- 2.- Seleccionar "Obtener datos"
- 3.- Cargar los 3 archivos csv (Asignaturas.csv, estudiantes.csv y Incripcion.csv)
- 4.- BI mostrara una vista previa y podremos cambiar tipos de datos de columnas, eliminar filas o columnas
- 5.- En la vista de modelo de BI, podremos relacionar las tablas entre si, de esta forma relacionamos Inscripcion con estudiantes y asignaturas
- 6.- Una vez caragdos los datos, podremos crear visualizaciones

**Pregunta 4**

Describa que pasos usted usaría para retirar campos vacíos y caracteres especiales (comas, saltos
de línea, diagonales, pipelines, etc.) en un archivo CSV y depositarlo en un repositorio utilizando
una herramienta de flujos analíticos que usted conozca (Alteryx, Tableau Prep, etc.).

**Para Power BI**

- Con el editor de powerquery de Power BI hacienco clic en "Transformar datos", por ejemplo para quitar valores vaciones, seleccionamos la columna damos clic en la pestaña de reducir filas y le damos en quitar filas vacias

- Para caracteres especiales en la parte de "Reemplazar valores" , ingresamos el caracter especial que deseamos eliminar y dejamos el campo vacio de "Reemplazar con" 

- Para cambiar tipos de datos basta con seleccionar la columna y seleccionar el tipo de dato que deseamos, y le damos en "Cerrar y aplicar"